<a href="https://www.kaggle.com/code/aleksandrmorozov123/gpt-architecture?scriptVersionId=237576397" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**In this notebook I will create GPT (Generative Pretrained Transformer) small model and training this later**

In [2]:
from importlib.metadata import version

print("matplolib version: ", version("matplotlib"))
print("torch version: ", version("torch"))
print("tiktoken version: ", version("tiktoken"))

matplolib version:  3.7.5
torch version:  2.5.1+cu124
tiktoken version:  0.9.0


In [3]:
# configuration of the GPT model
GPT_CONFIG_124M = {
    "vocab_size": 50257,     # vocabulary size
    "context_length": 1024,  # context length
    "emb_dim": 768,          # embedding dimensions
    "n_heads": 12,           # number of attention heads
    "n_layers": 12,          # number of layers
    "drop_rate": 0.1,        # dropout rate
    "qkv_bias": False        # Query-Key-Value bias
}

In [4]:
import torch
import torch.nn as nn

class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # use a placeholder for Transformer Block
        self.trf_blocks = nn.Sequential(*[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        # use a placeholder for Layer Norm
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        return x 

class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()

    def forward(self, x):
        return x

In [5]:
# encoding process
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
batch = []

txt1 = "If you grind and "
txt2 = "You wake every day and"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[ 1532,   345, 20078,   290,   220],
        [ 1639,  7765,   790,  1110,   290]])


In [6]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)

logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 5, 50257])
tensor([[[-0.4086,  1.0408, -0.5351,  ..., -1.6850, -0.2098,  0.3704],
         [-0.1377,  1.9332, -0.4629,  ..., -0.4409, -0.0980,  0.0817],
         [ 0.2650, -0.0898, -0.0479,  ...,  0.4372,  0.3871,  0.2004],
         [-0.5968,  0.7088,  1.3957,  ...,  0.4451,  1.5210,  0.8915],
         [-0.7740, -0.9768,  1.2553,  ...,  0.7457, -0.4746,  0.2609]],

        [[-0.1776,  0.6227, -0.0074,  ..., -1.2598, -0.6497,  0.8217],
         [-0.6403,  1.4392, -1.2086,  ...,  0.9956,  1.0368,  2.2615],
         [-0.2581, -0.2354, -0.3046,  ..., -0.9638,  0.7429, -0.2973],
         [-0.0722,  0.5075,  0.0342,  ...,  1.2990,  0.3487,  0.2775],
         [-1.0624, -0.9751,  2.0261,  ..., -1.3601,  0.3677,  0.0639]]],
       grad_fn=<UnsafeViewBackward0>)


In [7]:
# layer normalization
torch.manual_seed(123)

# create 2 training examples with 5 dimensions (features) each
batch_example = torch.randn(2, 5) 

layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
out = layer(batch_example)
print(out)

tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)


In [8]:
mean = out.mean(dim=-1, keepdim=True)
var = out.var(dim=-1, keepdim=True)

print("Mean:\n", mean)
print("Variance:\n", var)

Mean:
 tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>)


In [9]:
out_norm = (out - mean) / torch.sqrt(var)
print("Normalized layer outputs:\n", out_norm)

mean = out_norm.mean(dim=-1, keepdim=True)
var = out_norm.var(dim=-1, keepdim=True)
print("Mean:\n", mean)
print("Variance:\n", var)

Normalized layer outputs:
 tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)
Mean:
 tensor([[9.9341e-09],
        [1.9868e-08]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


In [10]:
torch.set_printoptions(sci_mode=False)
print("Mean:\n", mean)
print("Variance:\n", var)

Mean:
 tensor([[    0.0000],
        [    0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)
